半决赛（4强）队伍次数统计

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("local[*]").getOrCreate()
summary = spark.read.format("csv").option("header",True).option("encoding","utf-8").option("sep",",").load("../dataset/WorldCupsSummary.csv")
summary.createTempView("summary_tb")

In [3]:
summary.printSchema()

root
 |-- Year: string (nullable = true)
 |-- HostCountry: string (nullable = true)
 |-- Winner: string (nullable = true)
 |-- Second: string (nullable = true)
 |-- Third: string (nullable = true)
 |-- Fourth: string (nullable = true)
 |-- GoalsScored: string (nullable = true)
 |-- QualifiedTeams: string (nullable = true)
 |-- MatchesPlayed: string (nullable = true)
 |-- Attendance: string (nullable = true)
 |-- HostContinent: string (nullable = true)
 |-- WinnerContinent: string (nullable = true)


In [4]:
semifinal_number = spark.sql('''
SELECT IFNULL(IFNULL(t1.country,t2.country),IFNULL(t3.country,t4.country)) AS country,IFNULL(winner_counts,0)+IFNULL(second_counts,0)+IFNULL(third_counts,0)+IFNULL(fourth_counts,0) AS counts  FROM 
(SELECT  Winner AS country,COUNT(Winner) AS winner_counts FROM summary_tb
GROUP BY Winner) t1
FULL OUTER JOIN (
SELECT  Second AS country,COUNT(Second)AS second_counts FROM summary_tb
GROUP BY Second) t2 ON t1.country=t2.country
FULL OUTER JOIN (
SELECT  Third AS country,COUNT(Third)AS third_counts FROM summary_tb
GROUP BY Third ) t3 ON t1.country = t3.country
FULL OUTER JOIN (
SELECT  Fourth AS country,COUNT(Fourth)AS fourth_counts FROM summary_tb
GROUP BY Fourth) t4 ON t1.country = t4.country
ORDER BY counts DESC
''').show()

+--------------+------+
|       country|counts|
+--------------+------+
|        Brazil|    11|
|    Germany FR|     8|
|         Italy|     8|
|        France|     6|
|     Argentina|     5|
|       Germany|     5|
|       Uruguay|     5|
|   Netherlands|     3|
|       England|     3|
|Czechoslovakia|     2|
|       Hungary|     2|
|        Poland|     2|
|        Sweden|     2|
|         Spain|     2|
|    Yugoslavia|     2|
|       Croatia|     1|
|        Sweden|     1|
|       Austria|     1|
|       Belgium|     1|
|         Chile|     1|
+--------------+------+


In [5]:
semifinal_number_2 = spark.sql('''
SELECT country,SUM(counts) AS counts FROM
(SELECT  Winner AS country,COUNT(Winner) AS counts FROM summary_tb
GROUP BY country
UNION ALL
SELECT Second AS country,COUNT(Second) AS counts FROM summary_tb
GROUP BY country
UNION ALL
SELECT Third AS country,COUNT(Third) AS counts FROM summary_tb
GROUP BY country
UNION ALL
SELECT Fourth AS country,COUNT(Fourth) AS counts FROM summary_tb
GROUP BY country) t GROUP BY country ORDER BY counts DESC
''')

In [7]:
semifinal_number_2.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/worldcup") \
    .option("dbtable", "semifinal_number") \
    .option("user", "root") \
    .option("password", "") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .mode("append") \
    .save()
semifinal_number_2.show()

+--------------+------+
|       country|counts|
+--------------+------+
|        Brazil|    11|
|         Italy|     8|
|    Germany FR|     8|
|        France|     6|
|       Germany|     5|
|     Argentina|     5|
|       Uruguay|     5|
|   Netherlands|     5|
|        Sweden|     4|
|       England|     3|
|         Spain|     2|
|Czechoslovakia|     2|
|       Croatia|     2|
|       Hungary|     2|
|       Belgium|     2|
|        Poland|     2|
|      Portugal|     2|
|       Austria|     2|
|    Yugoslavia|     2|
|        Turkey|     1|
+--------------+------+
